## Building Azure AI Agent with Plugins

![azure-ai-agent-with-plugins](./Assets/AzureAIAgentWithPlugins.png)

### Installing dependencies and libraries

In [ ]:
%pip install semantic-kernel==1.28.1, azure-identity, azure-ai-projects==1.0.0b7

### Setting the Environment Variables

In [ ]:
import os

os.environ["AZURE_CLIENT_ID"] = ""
os.environ["AZURE_TENANT_ID"] = ""
os.environ["AZURE_CLIENT_SECRET"] = ""


### Loading our OpenAPI and Code Interpreter Tool

In [ ]:
from azure.identity import DefaultAzureCredential
from semantic_kernel.agents import AzureAIAgent, AzureAIAgentSettings
import os
from azure.ai.projects import AIProjectClient
from azure.ai.projects.models import MessageTextContent
from azure.ai.projects.models import OpenApiTool, OpenApiAnonymousAuthDetails
import asyncio
from typing import Any, Callable, Set, Dict, List, Optional
import jsonref
from azure.ai.projects.models import FunctionTool, ToolSet
import json
from azure.ai.projects.models import CodeInterpreterTool, MessageAttachment

model = ""

project_client = AzureAIAgent.create_client(credential=DefaultAzureCredential(),
                           conn_str=""
)

with open("./weather_openapi.json", "r") as f:
    openapi_spec = jsonref.loads(f.read())

# Create Auth object for the OpenApiTool 
auth = OpenApiAnonymousAuthDetails()

# Initialize agent OpenApi tool using the read in OpenAPI spec
openapi = OpenApiTool(
    name="get_weather", spec=openapi_spec, description="Retrieve weather information for a location", auth=auth
)

code_interpreter = CodeInterpreterTool()

### Encapsulating both Tools in a Toolset

In [ ]:
toolset = ToolSet()
toolset.add(openapi)
toolset.add(code_interpreter)

### Creating Azure AI Agent (in Cloud)

In [ ]:
agent = await project_client.agents.create_agent(
        model=model,
        name="multiple-tools-assistant",
        instructions="You are a helpful assistant",
        toolset=toolset,
    )
# [END create_agent_toolset]
print(f"Created agent, ID: {agent.id}")
agent_id = agent.id

thread = await project_client.agents.create_thread()
print(f"Created thread, thread ID: {thread.id}")
thread_id = thread.id

### Creating Azure AI Agent (in Semantic Kernel ~ Locally)

In [ ]:
agent_definition = await project_client.agents.get_agent(agent_id=agent_id)
agent = AzureAIAgent(client=project_client, definition=agent_definition)

### Invoking the Agent with a Thread

In [ ]:
from semantic_kernel.agents import AzureAIAgentThread

thread: AzureAIAgentThread = AzureAIAgentThread(client=project_client, thread_id=thread_id)
print(f"Working with the thread, ID: {thread.id}")


user_input = "What is the weather in Paris and generate a chart for the same?"
# 3. Call the agent with user input
async def get_response_from_agent():
    response =  await agent.get_response(
        messages = user_input,
        thread=thread,
    )
    
    return response

response = await get_response_from_agent()
print(response)

### Fetching the PNG file of the Chart

In [ ]:
from azure.ai.projects.models import CodeInterpreterTool, MessageAttachment
from azure.ai.projects.models import FilePurpose, MessageRole
from pathlib import Path


messages = await project_client.agents.list_messages(thread_id=thread_id)
print(f"Messages: {messages}")

last_msg = messages.get_last_text_message_by_role(MessageRole.AGENT)
if last_msg:
        print(f"Last Message: {last_msg.text.value}")

for image_content in messages.image_contents:
        print(f"Image File ID: {image_content.image_file.file_id}")
        file_name = f"{image_content.image_file.file_id}_image_file.png"
        await project_client.agents.save_file(file_id=image_content.image_file.file_id, file_name=file_name)
        print(f"Saved image file to: {Path.cwd() / file_name}")